In [33]:
# Importaciones e instalaciones necesarias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver-manager

# Para trabajar con mongodb
!pip install pymongo
from pymongo import MongoClient
client = MongoClient('localhost')
db = client['pruebaTusDatos']
col = db['procesos']

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/d8/ff/7b7e9afdca4dd46ef0fba00d023daab442144c1ed63c068ccb9e04738727/pymongo-4.7.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/87/a1/8c5287991ddb8d3e4662f71356d9656d91ab3a36618c3dd11b280df0d255/dnspython-2.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/483.8 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/483.8 kB 1.1 MB/s eta 0:00:01
   ----------- ---------------------------- 143.4/483.8 kB 1.7 MB/s eta 0:00:01
   ---------------------- ----------------- 276.5/483.8 kB 2.1 MB/s eta 0:00:01
   --------------------------------- ------ 399.4/483.8 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 483.8/483.8 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ------------------------- ----

In [40]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Conexión con la página web a "scrapear"
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=opts)
driver.get('https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros')

type_process = int(input("Desea ingresar código para: \n"+
                   "1. Demandante\n"+
                   "2. Demandado\n"))
# Código del demandante o demandado
# Demandante1 = 0968599020001
# Demandante2 = 0992339411001

# Demandado1 = 1791251237001
# Demandado2 = 0968599020001
codigo = input("Ingrese el número de identificación: ")



Desea ingresar código para: 
1. Demandante
2. Demandado
1
Ingrese el número de identificación: 0992339411001


In [41]:
# Me doy cuenta que la pagina carga el formulario dinamicamente luego de que la carga incial ha sido completada
# Por eso tengo que esperar que aparezca 
if type_process == 1:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaActor']")))
    input_user.send_keys(codigo)
elif type_process == 2:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaDemandado']")))
    input_user.send_keys(codigo)

# Se obtiene dato para distinguir entre procesos de demandado y demandante


# Click al boton buscar
boton_buscar = driver.find_element(By.XPATH,"//button[@type = 'submit']")
boton_buscar.click()


# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

In [42]:
# Se hace un recorrido del listado de procesos para poder iterar entre sus elementos y obtener:
#    id_proceso 
#    fecha 
#    numero_proceso 
#    accion_infraccion 
for proceso in listado_procesos:
    id_proceso = proceso.find_element('xpath','.//div[@class="id"]').text
    fecha = proceso.find_element('xpath','.//div[@class="fecha"]').text
    numero_proceso = proceso.find_element('xpath','.//div[@class="numero-proceso"]').text
    accion_infraccion = proceso.find_element('xpath','.//div[@class="accion-infraccion"]').text
    
    # Se envia la información obtenida a la base de datos
    col.insert_one({
        'id':id_proceso,
        'fecha': fecha,
        'numero_proceso': numero_proceso,
        'accion_infraccion': accion_infraccion
    })
    